In [178]:
import numpy as np

In [179]:
# define some activation and error functions
def tanh(x, derivative=False):
    """Implements the hyperbolic tangent function element wise over an array x.

    Parameters
    ----------
    x : numpy array
        This array contains arguments for the hyperbolic tangent function.
    derivative : bool
        Indicates whether to use the hyperbolic tangent function or its derivative.

    Returns
    -------
    numpy array
        An array of equal shape to `x`.
    """

    if derivative:
        tanh_not_derivative = tanh(x)
        return 1.0 - tanh_not_derivative**2
        #return 1.0 - x**2
    else:
        return np.tanh(x)

def relu(x, derivative=False):
    if derivative:
        return 1 * (x > 0)  #returns 1 for any x > 0, and 0 otherwise
    
    return np.maximum(0, x)

def softmax(x, derivative=False):
    if derivative:
        pass
    
    return np.exp(x)/sum(np.exp(x)) 

def mean_squared_error(target_output, actual_output, derivative=False):
    try:
        assert(target_output.shape == actual_output.shape)
    except AssertionError:
        print(f"Shape of target vector: {target_output.shape} does not match shape of actual vector: {actual_output.shape}")

    if not derivative:
        error = np.sum(0.5 * np.sum((target_output-actual_output)**2, axis=1, keepdims=True))
    else:
        error = (actual_output - target_output)

    return error

In [327]:
class NeuralNet(object):
    RNG = np.random.default_rng()


    def __init__(self, 
                 topology:list[int] = [], 
                 learning_rate = 0.001,
                 momentum      = 0.1,
                 hidden_activation_func=relu, 
                 output_activation_func=relu,
                 init_method='random'):
        
        self.topology    = topology
        self.weight_mats = []
        self.bias_mats   = []   #will hold the weights for the bias nodes

        self.learning_rate = learning_rate
        self.momentum      = momentum

        self.hidden_activation = hidden_activation_func
        self.output_activation = output_activation_func

        

        self._init_weights_and_biases()
        self.size    = len(self.weight_mats)
        self.netIns  = [None] * self.size
        self.netOuts = [None] * self.size
        self.last_change = [np.zeros(mat.shape) for mat in self.weight_mats]

        self._init_gradients()

    def _init_weights_and_biases(self, method='random'):
        #-- decide which initialization method to use. I added some of the popular ones
        if method.lower() == 'random':
            _init_func = lambda num_rows, num_cols: self.RNG.random(size=(num_rows, num_cols))

        elif method.lower() == 'xavier':
            _init_func = self._xavier_weight_initialization

        else:
            print(f"\t-> initialization method {method} not recognized. Defaulting to 'random'")
            _init_func = lambda num_rows, num_cols: self.RNG.random(size=(num_rows, num_cols))

        #-- set up matrices
        if len(self.topology) > 1:
            j = 1
            for i in range(len(self.topology)-1):
                num_rows = self.topology[i]
                num_cols = self.topology[j]

                mat         = _init_func(num_rows, num_cols)  #the +1 accounts for the bias weights
                bias_vector = _init_func(1, num_cols) 

                self.weight_mats.append(mat)
                self.bias_mats.append(bias_vector)

                j += 1
    
    def _xavier_weight_initialization(self, num_rows, num_cols):
        '''A type of weight initialization that seems to be tailored to sigmoidal activation functions.
        Here is a reference: https://machinelearningmastery.com/weight-initialization-for-deep-learning-neural-networks/'''
        num_inputs = self.topology[0]

        lower_bound = -1 / np.sqrt(num_inputs)
        upper_bound = 1 / np.sqrt(num_inputs)

        mat = self.RNG.uniform(lower_bound, upper_bound, (num_rows, num_cols))
        return mat
    
    def _init_gradients(self,):
        self.stored_gradients = [None] * len(self.weight_mats)
                
    
    def OLD_feedforward(self, input_vector):

        self.netIns = []
        self.netOuts = []

        I = input_vector  #rename vector to match typical nomenclature
        
        
        # print(f"I before after append: {I.shape}")

        # I = np.concatenate((I, np.ones((I.shape[0], 1))), axis = 1)                # adds the bias input of 1        
        # print(f"I shape after append: {I.shape}")
        # self.netOuts.append(I)

        for idx, W in enumerate(self.weight_mats):
            
            #bias_vector = self.bias_mats[idx]

            #-- add bias constant of 1 to input matrix
            I = np.concatenate((I, np.ones((I.shape[0], 1))), axis = 1)
            self.netOuts.append(I)
            # if idx == 0:
            #     self.netOuts.append(I)    #I need to store the outputs of the first layer before anything is done to it

            print(f"Weight shape in layer {idx} is: {W.shape}")

            I = np.dot(I, W) #+ bias_vector
            self.netIns.append(I)
            
            #-- apply activation function
            if idx == len(self.weight_mats) - 1:
                out_vector = self.output_activation(I)  #output layer
            else:
                I          = self.hidden_activation(I)  #hidden layers
                # self.netOuts.append(I)
            
        return out_vector
    
    def feedforward(self, input_vector):

        self.netIns.clear()
        self.netOuts.clear()

        I = input_vector  #rename vector to match typical nomenclature
        
        
        # print(f"I before after append: {I.shape}")

        # I = np.concatenate((I, np.ones((I.shape[0], 1))), axis = 1)                # adds the bias input of 1        
        # print(f"I shape after append: {I.shape}")
        # self.netOuts.append(I)

        for idx, W in enumerate(self.weight_mats):
            
            bias_vector = self.bias_mats[idx]

            #-- add bias constant of 1 to input matrix
            #I = np.concatenate((I, np.ones((I.shape[0], 1))), axis = 1)
            self.netOuts.append(I)
            # if idx == 0:
            #     self.netOuts.append(I)    #I need to store the outputs of the first layer before anything is done to it

            # print(f"Weight shape in layer {idx} is: {W.shape}")

            I = np.dot(I, W) + bias_vector
            self.netIns.append(I)
            
            #-- apply activation function
            if idx == len(self.weight_mats) - 1:
                out_vector = self.output_activation(I)  #output layer
            else:
                I          = self.hidden_activation(I)  #hidden layers
                # self.netOuts.append(I)
            
        return out_vector
    
    
    def _gradient_descent(self, gradients,):
        """Uses the gradients computed by the backpropagation method to update network weights.

        performs stochastic gradient descent and adjusts the weights


        Parameters
        ----------
        gradients : python iterable
            This iterable {list, tuple, etc.} contains numpy arrays.
            Each numpy array is the gradient matrix computed by backpropagation for each layer matrix.

        """
        
        for layer_idx in range(self.size):
            delta_weight = self.learning_rate * gradients[layer_idx]

            full_change = delta_weight + (self.momentum * self.last_change[layer_idx])

            self.weight_mats[layer_idx] -= full_change
            self.last_change[layer_idx] = 1 * gradients[layer_idx] #copy gradient mat


    def OLD_backprop(self, 
                 target,
                 output,
                 error_func,):
        """Backpropagation.

        Parameters
        ----------
        input_samples : numpy array
            Contains all samples in a batch.
        target_outputs : numpy array
            Matching targets for each sample in `input_samples`.
        output : numpy array
            Actual output from feedforward propagation. It will be used to check the network's error.
        error_func : function object
            This is the function that computes the error of the epoch and used during backpropagation.
            It must accept parameters as: error_func(target={target numpy array},actual={actual output from network},derivative={boolean to indicate operation mode})
        hidden_activation : function object, optional
            It is the activation function for hidden layers. It must be able to accept numpy arrays.
            It must also provide a parameter to indicate operation in derivative or normal mode.
        output_activation : function object, optional
            It is the activation function for final layer. It must be able to accept numpy arrays.
            It must also provide a parameter to indicate operation in derivative or normal mode.

        """

        #Compute gradients and deltas
        for i in range(self.size):
            back_index =self.size-1 -i                  # This will be used for the items to be accessed backwards
            if i!=0:  #hidden layers
                W_trans = self.weight_mats[back_index+1].T        #we use the transpose of the weights in the current layer
                d_activ = self.hidden_activation(self.netIns[back_index],derivative=True)
                d_error = np.dot(delta, W_trans)
                delta = d_error * d_activ
                gradient_mat = np.dot(self.netOuts[back_index].T , delta)
                self.stored_gradients[back_index] = gradient_mat
            else:
                #Here we calculate gradients for final layer
                d_activ = self.output_activation(self.netIns[back_index], derivative=True)
                d_error = error_func(target,output,derivative=True)
                delta = d_error * d_activ

                gradient_mat = np.dot(self.netOuts[back_index].T , delta)

                self.stored_gradients[back_index] = gradient_mat

        # Update weights using the computed gradients
        self._gradient_descent(gradients=self.stored_gradients,)


    def backprop(self, 
                 target,
                 output,
                 error_func,):
        """Backpropagation.

        Parameters
        ----------
        input_samples : numpy array
            Contains all samples in a batch.
        target_outputs : numpy array
            Matching targets for each sample in `input_samples`.
        output : numpy array
            Actual output from feedforward propagation. It will be used to check the network's error.
        error_func : function object
            This is the function that computes the error of the epoch and used during backpropagation.
            It must accept parameters as: error_func(target={target numpy array},actual={actual output from network},derivative={boolean to indicate operation mode})
        hidden_activation : function object, optional
            It is the activation function for hidden layers. It must be able to accept numpy arrays.
            It must also provide a parameter to indicate operation in derivative or normal mode.
        output_activation : function object, optional
            It is the activation function for final layer. It must be able to accept numpy arrays.
            It must also provide a parameter to indicate operation in derivative or normal mode.

        """

        #Compute gradients and deltas
        for i in range(self.size):
            back_index =self.size-1 -i                  # This will be used for the items to be accessed backwards
            if i!=0:  #hidden layers
                W_trans = self.weight_mats[back_index+1].T        #we use the transpose of the weights in the current layer
                d_activ = self.hidden_activation(self.netIns[back_index],derivative=True)
                d_error = np.dot(delta, W_trans)
                delta = d_error * d_activ
                gradient_mat = np.dot(self.netOuts[back_index].T , delta)
                self.stored_gradients[back_index] = gradient_mat
            else:
                #Here we calculate gradients for final layer
                d_activ = self.output_activation(self.netIns[back_index], derivative=True)
                d_error = error_func(target,output,derivative=True)
                delta = d_error * d_activ

                gradient_mat = np.dot(self.netOuts[back_index].T , delta)

                self.stored_gradients[back_index] = gradient_mat

        # Update weights using the computed gradients
        self._gradient_descent(gradients=self.stored_gradients,)


    def train(self, input_set, target_set, epochs=1000, error_threshold=1E-10, error_func=mean_squared_error):

        #improve this part later
        inputs = input_set
        targets = target_set

        for i in range(epochs):
            nnet_output = self.feedforward(inputs)
            error = error_func(targets, nnet_output)

            if error <= error_threshold:
                print(f"\t-> error {error} is lower than threshold {error_threshold}")
                break

            self.backprop(target=targets, output=nnet_output, error_func=error_func,)

            

In [328]:
# Create a neural network for the XOR logic gate, so two inputs and one output neuron. 
nnet = NeuralNet([2,3,1], hidden_activation_func=relu, output_activation_func=tanh, init_method='random')

# XOR logic gate truth table (this will be our training set)
inputs  = np.array([[0,0], 
                    [0,1], 
                    [1,0], 
                    [1,1]])

training_targets = {
    'XOR'  : np.array([[0],
                       [1],
                       [1],
                       [0]]),

    'OR'   : np.array([[0],
                       [1],
                       [1],
                       [1]]),

    'AND'  : np.array([[0],
                       [0],
                       [0],
                       [1]]),

    'NAND' : np.array([[1],
                       [1],
                       [1],
                       [0]]),
}

In [329]:
nnet.train(inputs, training_targets['NAND'], 1000)

In [330]:
nnet.netOuts

[array([[0, 0],
        [0, 1],
        [1, 0],
        [1, 1]]),
 array([[0.43838377, 0.68767519, 0.78711081],
        [1.1709636 , 1.35595618, 1.47769556],
        [0.97890992, 1.60443311, 1.6874323 ],
        [1.71148975, 2.27271409, 2.37801706]])]

In [331]:
nnet.feedforward(inputs[0])

array([[0.96756751]])

In [332]:
inputs[2]

array([1, 0])

In [333]:
x = np.array([0, -0.3, 2, 0.5])

In [334]:
for gate in training_targets:
    print(f"\n\n{'='*40}\nTraining {gate} gate:\n{'='*40}\n")
    input_data = inputs 
    target_data = training_targets[gate]

    nnet = NeuralNet([2, 3, 1], hidden_activation_func=tanh, output_activation_func=tanh, init_method='xavier')
    nnet.train(input_data, target_data, epochs=1000, error_func=mean_squared_error)

    # Test different inputs
    for i in range(len(input_data)):
        output = nnet.feedforward(input_data[i])
        print(f"Testing Network:\n\tinput vector   : {input_data[i]}\n\toutput vector  : {output}\n\texpected output: {target_data[i]}")
    



Training XOR gate:

Testing Network:
	input vector   : [0 0]
	output vector  : [[-0.13343711]]
	expected output: [0]
Testing Network:
	input vector   : [0 1]
	output vector  : [[0.91991397]]
	expected output: [1]
Testing Network:
	input vector   : [1 0]
	output vector  : [[0.91615162]]
	expected output: [1]
Testing Network:
	input vector   : [1 1]
	output vector  : [[0.10217606]]
	expected output: [0]


Training OR gate:

Testing Network:
	input vector   : [0 0]
	output vector  : [[0.00567196]]
	expected output: [0]
Testing Network:
	input vector   : [0 1]
	output vector  : [[0.98236242]]
	expected output: [1]
Testing Network:
	input vector   : [1 0]
	output vector  : [[0.96381239]]
	expected output: [1]
Testing Network:
	input vector   : [1 1]
	output vector  : [[0.97916891]]
	expected output: [1]


Training AND gate:

Testing Network:
	input vector   : [0 0]
	output vector  : [[-0.17852645]]
	expected output: [0]
Testing Network:
	input vector   : [0 1]
	output vector  : [[0.139800

In [234]:
def softmax_grad(softmax):
    # Reshape the 1-d softmax to 2-d so that np.dot will do the matrix multiplication
    s = softmax.reshape(-1,1)
    return np.diagflat(s) - np.dot(s, s.T)

In [250]:
x

array([ 0. , -0.3,  2. ,  0.5])

In [311]:
[None] * 3

[None, None, None]